In [1]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
import pandas as pd

In [2]:
# Get authentication credentials to initialize ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VMjw30wmz_fN5hz_kiUz6y5BNZAH5U6GmInt1XDnrQTq7uClUyJsh4

Successfully saved authorization token.


In [3]:
# Define geometry for data collection


# Nanda Bet larger S1 extent
# roi = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271]);
# shp_file = 'data/s1_boundary.shp'
# boundary = geemap.shp_to_ee(shp_file)


# Nanda Bet smaller region
roi = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  
shp_file = 'data/smaller_region.shp'
boundary = geemap.shp_to_ee(shp_file)

In [29]:
# Get MODIS Land Surface Temperature in ROI

d1 = '2020-03-01'
d2 = '2020-03-02'

dataset = ee.ImageCollection('MODIS/061/MOD11A1').filter(ee.Filter.date(d1, d2)).filterBounds(boundary.geometry())                 
landSurfaceTemperature = dataset.select('LST_Day_1km').mean().clip(boundary.geometry())

lst_c = landSurfaceTemperature.multiply(0.02).add(-273.15)
lst_f = lst_c.multiply(1.8).add(32)


In [30]:
print(lst_f)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.add",
    "arguments": {
      "image1": {
        "functionInvocationValue": {
          "functionName": "Image.multiply",
          "arguments": {
            "image1": {
              "functionInvocationValue": {
                "functionName": "Image.add",
                "arguments": {
                  "image1": {
                    "functionInvocationValue": {
                      "functionName": "Image.multiply",
                      "arguments": {
                        "image1": {
                          "functionInvocationValue": {
                            "functionName": "Image.clip",
                            "arguments": {
                              "geometry": {
                                "functionInvocationValue": {
                                  "functionName": "Collection.geometry",
                                  "arguments": {
                                    "collection"

In [31]:
# Define visualization parameters

precip_vis = {
  'bands': ['total_precipitation'],
  'min': 0,
  'max': 0.2,
  'palette': [
    "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
    "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
    "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
  ],
  'opacity': 0.5
};


landSurfaceTemperatureVis = {
  'min': 13000.0,
  'max': 16500.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
  'opacity': 0.5,
};

lst_f_vis = {
  'min': 85.0,
  'max': 120.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
  'opacity': 0.5,
};

lst_c_vis = {
  'min': 29.0,
  'max': 48.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
  'opacity': 0.5,
};


In [32]:
# Create map and add layers

M = geemap.Map()
M.centerObject(roi, 12)
M.addLayer(lst_c,lst_c_vis,'Precipitation')
M

Map(center=[23.595422252032954, 71.15205312407748], controls=(WidgetControl(options=['position', 'transparent_…

In [33]:
# Export modis lst

task_id = []
    
txt = 'modis_lst_c_mean_nanda_bet_' + d1
task = ee.batch.Export.image.toDrive(image=lst_c,
                                     description=txt,
                                     region=boundary.geometry(),
                                     fileNamePrefix=txt,
                                     scale=1000,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=1e9)
task.start()
task_id.append(task.id)
        

In [34]:
task.list()

[<Task S6WXAVQMCLMOQN5ZCQCUUUNL EXPORT_IMAGE: modis_lst_c_mean_nanda_bet_2020-03-01 (RUNNING)>,
 <Task 766IXR3MHZG5EURDEFK3XKJB EXPORT_IMAGE: modis_lst_c_mean_nanda_bet_2020-03-01_2020-03-03 (COMPLETED)>,
 <Task K7TRTC55AUN5BO5HI2SF32OU EXPORT_IMAGE: modis_lst_c_mean_nanda_bet_2020-03-01_2020-03-03 (COMPLETED)>,
 <Task H6A2X7ZV4JUB2WTWMHOEG2UG EXPORT_IMAGE: modis_lst_c_mean_nanda_bet_2020-03-01_2020-03-03 (COMPLETED)>,
 <Task 5CPK322R6SI5ZAK4PUXPHJUW EXPORT_IMAGE: modis_lst_c_mean_nanda_bet_2020-03-01_2020-03-03 (COMPLETED)>]